# Data manipulation with the ORM

In [30]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy import create_engine, MetaData, insert, select
from sqlalchemy import Table, Column, Integer, String, ForeignKey

In [15]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
session = Session(engine)

In [16]:
# The Declarative Base refers to a MetaData collection that is created for us automatically
class Base(DeclarativeBase):
    pass

# MetaData collection is accessible via the DeclarativeBase.metadata class-level attribute.
Base.metadata

MetaData()

In [17]:
# Declarative table configuration
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str | None]
    addresses: Mapped[list["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [18]:
Base.metadata.create_all(engine)

2024-08-13 21:27:46,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 21:27:46,537 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-08-13 21:27:46,538 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,539 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-08-13 21:27:46,540 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,541 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-08-13 21:27:46,542 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,543 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-08-13 21:27:46,544 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,546 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-13 21:27:46,547 INFO sqlalchemy.engine.Engine [no key 0.00119s] ()
2024-08-13 21:27:46,548 INFO sqlalchemy.engine.

In [32]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")
sandy = User(name="sandy", fullname="Sandy Cheeks")

squidward, krabs, sandy

(User(id=None, name='squidward', fullname='Squidward Tentacles'),
 User(id=None, name='ehkrabs', fullname='Eugene H. Krabs'),
 User(id=None, name='sandy', fullname='Sandy Cheeks'))

To illustrate the addition process step by step, we will create a Session without using a context manager (and hence we must make sure we close it later!):

In [33]:
# The objects are then added to the Session using the Session.add() method.
# When this is called, the objects are in a state known as pending and have not been inserted yet:
session.add(squidward)
session.add(krabs)
session.add(sandy)


In [34]:
# When we have pending objects, we can see this state by
# looking at a collection on the Session called Session.new:
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs'), User(id=None, name='sandy', fullname='Sandy Cheeks')])

In [35]:
# The Session makes use of a pattern known as unit of work.
# This generally means it accumulates changes one at a time, but does not actually communicate them to the database until needed.
# This allows it to make better decisions about how SQL DML should be emitted in the transaction based on a given set of pending changes.
# When it does emit SQL to the database to push out the current set of changes, the process is known as a flush.
session.flush()

2024-08-13 21:38:59,504 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-13 21:38:59,504 INFO sqlalchemy.engine.Engine [cached since 672.9s ago (insertmanyvalues) 1/3 (ordered; batch not supported)] ('squidward', 'Squidward Tentacles')
2024-08-13 21:38:59,511 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-13 21:38:59,511 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('ehkrabs', 'Eugene H. Krabs')
2024-08-13 21:38:59,513 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-13 21:38:59,513 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('sandy', 'Sandy Cheeks')


Above we observe the Session was first called upon to emit SQL, so it created a new transaction and emitted the appropriate INSERT statements for the two objects. The transaction now remains open until we call any of the Session.commit(), Session.rollback(), or Session.close() methods of Session.

It is usually **unnecessary** as the Session features a behavior known as autoflush, which we will illustrate later. It also flushes out changes whenever Session.commit() is called.

In [23]:
# Autogenerated primary key attributes
squidward.id, krabs.id

(1, 2)

In [36]:
session.commit()

2024-08-13 21:39:02,621 INFO sqlalchemy.engine.Engine COMMIT


In [37]:
# Manually closing the session
session.close()

2024-08-13 21:39:06,315 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# entities are detached after the session is closed
# squidward.id, krabs.id # DetachedInstanceError

In [40]:
sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()
sandy

2024-08-13 21:40:06,931 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-13 21:40:06,932 INFO sqlalchemy.engine.Engine [cached since 148.5s ago] ('sandy',)


User(id=5, name='sandy', fullname='Sandy Cheeks')

In [41]:
sandy in session.dirty

False

In [42]:
sandy_fullname = session.execute(select(User.fullname).where(User.id == 5)).scalar_one()
print(sandy_fullname)

2024-08-13 21:40:55,682 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2024-08-13 21:40:55,684 INFO sqlalchemy.engine.Engine [generated in 0.00132s] (5,)
Sandy Cheeks


In [43]:
patrick = session.get(User, 3)

2024-08-13 21:42:52,649 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2024-08-13 21:42:52,652 INFO sqlalchemy.engine.Engine [generated in 0.00259s] (3,)


In [44]:
patrick

User(id=3, name='squidward', fullname='Squidward Tentacles')

In [45]:
# Current ORM behavior is that patrick stays in the Session until the flush proceeds,
# which as mentioned before occurs if we emit a query:
session.delete(patrick)

In [46]:
session.execute(select(User).where(User.name == "patrick")).first()

2024-08-13 21:44:02,575 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-08-13 21:44:02,576 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (3,)
2024-08-13 21:44:02,578 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-08-13 21:44:02,579 INFO sqlalchemy.engine.Engine [generated in 0.00086s] (3,)
2024-08-13 21:44:02,580 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-13 21:44:02,581 INFO sqlalchemy.engine.Engine [cached since 384.2s ago] ('patrick',)


Above, the SELECT we asked to emit was preceded by a DELETE, which indicated the pending deletion for patrick proceeded. There was also a SELECT against the address table, which was prompted by the ORM looking for rows in this table which may be related to the target row; this behavior is part of a behavior known as cascade,

In [48]:
patrick in session

False

In [49]:
# rollback will not only roll back the transaction but also expire all objects currently associated with this Session
session.rollback()

2024-08-13 21:48:19,783 INFO sqlalchemy.engine.Engine ROLLBACK


In [52]:
# accessing an attribute again will autobegin a new transaction and refresh sandy with the current database row:
patrick.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x248a2b1f710>,
 'id': 3,
 'fullname': 'Squidward Tentacles',
 'name': 'squidward'}

In [53]:
session.close()

2024-08-13 21:49:39,007 INFO sqlalchemy.engine.Engine ROLLBACK


**Closing a session**

It **expunges** all objects from the Session.

This means that all the Python objects we had loaded for this Session, like sandy, patrick and squidward, are now in a state known as detached. In particular, we will note that objects that were still in an expired state, for example due to the call to Session.commit(), are now non-functional, as they don’t contain the state of a current row and are no longer associated with any database transaction in which to be refreshed: